# Pixyzの確率分布の記述方法

ここではまず，Pixyzにおける確率モデルの実装方法について説明します．

In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

In [2]:
from pixyz.utils import print_latex

## 1. 深層ニューラルネットワークを用いない確率分布の定義

### 1.1 シンプルな確率分布の定義

ガウス分布を作るためには，`Normal`をインポートして，平均（loc）と標準偏差（scale）を定義します．

In [13]:
from pixyz.distributions import Normal

x_dim = 50
p1_nor_x = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=['x'], features_shape=[x_dim], name='p_{1}')

なお``var``には，変数の名前を設定します．ここでは`"x"`を設定しています．

また，features_shapeでは次元数を指定します．ここではfeatures_shapeが50となっていますから，50次元のサンプルを生成する形になります．

上記で定義したp1の情報は次のようにみることができます．

In [14]:
print(p1_nor_x.distribution_name) 
print(p1_nor_x.prob_text)

Normal
p_{1}(x)


distribution_nameでは，確率分布の名前を確認できます．

prob_textでは，確率分布の形をテキストで出力できます．ここでテキストに書かれている確率変数は，上記のvarで指定したものです.

また，p1を丸ごとprintすると，以下のように表示されます．

In [15]:
print(p1_nor_x)

Distribution:
  p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


print_latexを利用するとLaTex表記で定義した確率分布が表示されます

In [16]:
print_latex(p1_nor_x)

<IPython.core.display.Math object>

次に，定義した分布からサンプリングしてみましょう． サンプリングは，`sample()`によって実行します．

In [17]:
p1_nor_x_samples = p1_nor_x.sample()
print(p1_nor_x_samples)
print(p1_nor_x_samples["x"])

{'x': tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,
          0.2152, -0.5242, -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -0.7479,
          1.1173,  0.2981]])}
tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,
          0.2152, -0.5242, -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -

出力はdict形式になっています．

サンプリング結果を確認したい変数について指定することで，中身を確認できます（ただし，この例では変数は"x"のみです）．

なお，サンプリング結果は，PyTorchのtensor形式になっています．

In [ ]:
###########尤度 Start

続いて，尤度の計算方法を説明します．
例えば，次のサンプルがあったとしましょう．

In [17]:
_sample1 = torch.Tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])

このサンプルにおける，p1の対数尤度を計算します．

これは，log_likelihoodによって簡単に計算できます．

In [30]:
log_like = p1.log_prob()
print_latex(log_like)

<IPython.core.display.Math object>

In [31]:
log_like.eval({'x': _sample1})

tensor([-72.5003])

なお，log_likelihoodの引数はdictで与えることに注意してください． これは，どの確率変数かを指定するためです（この例では，xしかありませんが）

In [ ]:
# 以下何をしたいのか書く
# 確率の積ができることを示したい

In [ ]:
############尤度 end

### 1.2 条件付確率分布の定義

次に条件付確率分布の定義の仕方を正規分布の例で見ていきます

正規分布ではパラメータは平均$\mu$と分散$\sigma^2$がありますが，今回は平均が条件付けられた正規分布を取り上げたいと思います

$p(x|\mu_{var}) = \cal N(x; \mu=\mu_{var}, \sigma^2=1)$

分布の条件付き変数の設定はcond_varで行います  
ここではmu_varという変数を正規分布の平均に設定したいため  
cond_var=['mu_var']  
loc='mu_var'  
とします

In [37]:
x_dim = 50
p1_nor_x__muvar = Normal(loc='mu_var', scale=torch.tensor(1.), var=['x'], cond_var=['mu_var'], features_shape=[x_dim])

In [36]:
print(p1_nor_x__mu)
print_latex(p1_nor_x__mu)

Distribution:
  p(x|\mu_{var})
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu_var'], input_var=['mu_var'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

これで平均が$\mu_{var}$で条件付けされる正規分布が定義できました  
試しに$\mu_{var}=0$としてxをサンプリングしてみます  
sampleメソッドにdict形式で変数を指定します

In [60]:
p1_nor_x__mu.sample({"mu_var": 0})

{'mu_var': 0,
 'x': tensor([[-2.3065e+00,  6.0366e-01,  1.7941e-01,  1.4465e-01, -3.5894e-01,
           4.7935e-01, -2.3828e-01, -1.3542e+00, -2.6925e-01,  1.3120e+00,
          -8.9197e-01, -7.5292e-01, -5.7266e-02,  2.2000e+00,  9.9124e-01,
          -5.8622e-02,  6.6930e-01,  6.5051e-01, -8.6923e-01,  8.8686e-01,
          -1.2211e+00, -6.7527e-01,  1.0476e+00, -3.1758e-01,  1.8568e-01,
          -2.7636e-01, -5.9385e-01, -3.0606e-01,  6.7713e-01,  6.8086e-01,
           1.9948e-01,  7.9927e-01, -1.5519e+00, -5.7648e-02,  7.4827e-02,
           3.5311e-01,  1.0170e+00,  9.3027e-01, -6.0679e-01, -5.2522e-01,
          -5.6619e-01,  6.6040e-04,  2.2788e+00,  4.7073e-01,  6.0782e-01,
          -5.8338e-02,  3.6000e-01,  4.6668e-01,  1.2831e+00,  1.2678e+00]])}

次に$\mu_{var}$自体も何らかの確率分布に従う変数とし，その確率分布を定めます  
ここでは仮にベルヌーイ分布とします  
$p(\mu_{var}) = \cal B(\mu_{var};p=0.3)$

In [57]:
from pixyz.distributions import Bernoulli
p2_ber_mu = Bernoulli(probs=torch.tensor(0.3), var=['mu_var'], features_shape=[x_dim])

In [58]:
print(p2_ber_mu)
print(p2_ber_mu.sample())
print_latex(p2_ber_mu)

Distribution:
  p(\mu_{var})
Network architecture:
  Bernoulli(
    name=p, distribution_name=Bernoulli,
    var=['mu_var'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (probs): torch.Size([1, 50])
  )
{'mu_var': tensor([[0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
         0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0.]])}


<IPython.core.display.Math object>

Pixyzでは分布の積は，掛け算で表すことができます  
定義した$p(\mu_{var})$と$p(x|\mu_{var})$を掛け合わせて同時分布$p(x, \mu_{var})$を定義します  
$p(x, \mu_{var}) = p(x|\mu_{var}) p(\mu_{var})$


In [59]:
p_joint_mu_x = p1_nor_x__muvar * p2_ber_mu
print(p_joint_mu_x) 
print_latex(p_joint_mu_x)

Distribution:
  p(x,\mu_{var}) = p(x|\mu_{var})p(\mu_{var})
Network architecture:
  Bernoulli(
    name=p, distribution_name=Bernoulli,
    var=['mu_var'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (probs): torch.Size([1, 50])
  )
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu_var'], input_var=['mu_var'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

同時分布でも今までと同様にsampleメソッドでサンプリングを行うことができます  
全ての変数とその値がdict形式で出力されます

In [62]:
p_joint_mu_x.sample()

{'mu_var': tensor([[0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
          0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0.,
          1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.]]),
 'x': tensor([[-2.3876e-01, -6.7961e-02,  8.4964e-01,  5.2330e-01, -2.1156e-01,
          -2.5420e-01, -3.6890e-02,  1.1820e+00, -1.7438e+00,  1.1289e-01,
           4.5267e-01,  3.1554e-01,  3.0990e-01, -2.8290e-01,  9.9063e-02,
           1.4938e+00,  6.7500e-01,  2.2910e+00,  2.8768e-01,  1.1313e+00,
          -1.9228e-03,  7.6455e-01,  1.4440e+00, -1.2114e-01,  1.1870e+00,
          -5.9569e-01, -1.5739e+00,  9.0095e-01,  1.0499e+00,  1.4293e+00,
           1.7927e+00, -1.1639e+00,  5.0744e-02, -1.4047e+00,  2.2090e+00,
           8.0967e-01,  6.2727e-01,  4.6669e-01, -2.7785e-01,  1.1217e-01,
          -7.8470e-01,  4.7250e-01,  1.8211e+00,  1.8643e+00,  4.9167e-01,
           4.4538e-01,  5.2116e-01,  7.5477e-01, -7.9745e-01,  9.0870e-01]])}

## 2 深層ニューラルネットワークと組み合わせた確率分布の設定

次に， 確率分布のパラメータを深層ニューラルネットワークで定義します．

例えば，ガウス分布の平均$\mu$と分散$\sigma^2$は， パラメータ$\theta$を持つ深層ニューラルネットワークによって，$\mu=f(x;\theta)$および$\sigma^2=g(x;\theta)$と定義できます．

したがって，ガウス分布は${\cal N}(\mu=f(x;\theta),\sigma^2=g(x;\theta))$となります．

$p(a) = {\cal N}(a; \mu=f(x;\theta),\sigma^2=g(x;\theta))$を定義してみましょう

Pixyzでは，次のようなクラスを記述することで，これを実現できます．

In [65]:
a_dim = 20

class ProbNorAgivX(Normal):
    """
    Probability distrituion Normal A given X
    p(a) = {\cal N}(a; \mu=f(x;\theta),\sigma^2=g(x;\theta)
    loc and scale are parameterized by theta given x
    """
    def __init__(self):
        super(ProbNorAgivX, self).__init__(cond_var=['x'], var=['a'])
        
        self.fc1 = nn.Linear(x_dim, 10)
        self.fc_loc = nn.Linear(10, a_dim)
        self.fc_scale = nn.Linear(10, a_dim)
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return {'loc': self.fc_loc(h1), 'scale': F.softplus(self.fc_scale(h1))}
p_nor_a__x = ProbNorAgivX()

まず， ガウス分布クラスを継承することで，ガウス分布のパラメータを深層ニューラルネットワークで定義することを明示します．

次に，コンストラクタで，利用するニューラルネットワークを記述します．これは，通常のPyTorchと同じです．

唯一異なる点は，superの引数にvarとcond_varの名前を指定している点です．

varは先程見たように，出力する変数の名前を指定します．一方，cond_varではニューラルネットワークの入力変数の名前を指定します．これは，ここで定義する分布において，条件付けられる変数とみなすことができます．

forwardについても，通常のPyTorchと同じです．ただし，注意点が2つあります．

* 引数の名前と数は，cond_varで設定したものと同じにしてください． 例えば，cond_var=["x", "y"]とした場合は，forward(self, x, y)としてください．ただし，引数の順番は変えても構いません．->引数の順番は変えても構いません．どういうこと
* 戻り値は，それぞれの確率分布のパラメータになります．上記の例ではガウス分布なので，平均と分散を指定しています．

そして最後に，定義した確率分布クラスのインスタンスを作成します．

次に，先程の例と同様，確率分布の情報を見てみましょう.

In [66]:
print(p_nor_a__x)
print_latex(p_nor_a__x)

Distribution:
  p(a|x)
Network architecture:
  ProbNorAgivX(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc_loc): Linear(in_features=10, out_features=20, bias=True)
    (fc_scale): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

p2の分布は，xで条件付けた形になっています．これらの表記は，superの引数で設定したとおりになっています．

次に，先程の例のように，サンプリングしてみましょう．

注意しなければならないのは，先ほどと異なり，条件づけた変数xがあるということです．

x_samplesをxとしてサンプリングしましょう．

In [67]:
x_samples = torch.Tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])

In [69]:
p_nor_a__x_samples = p_nor_a__x.sample({'x': x_samples})
print(p_nor_a__x_samples)
print(p_nor_a__x_samples['a'])
print(p_nor_a__x_samples['x'])

{'x': tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085, -0.2155,
          2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196, -0.5484, -0.3472,
          0.4730, -0.4286,  0.5514, -1.5474,  0.7575, -0.4068, -0.1277,  0.2804,
          1.7460,  1.8550, -0.7064,  2.5571,  0.7705, -1.0739, -0.2015, -0.5603,
         -0.6240, -0.9773, -0.1637, -0.3582, -0.0594, -2.4919,  0.2423,  0.2883,
         -0.1095,  0.3126, -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880,
         -1.1435, -0.6512]]), 'a': tensor([[-0.4467,  0.5744, -0.9273, -0.4159, -0.4318,  0.1391,  0.8793, -1.8171,
          1.7217,  0.3396, -0.6089, -1.0367,  1.0209,  0.3137, -0.3976, -0.7220,
         -0.2993,  0.9837,  0.4121,  0.3082]])}
tensor([[-0.4467,  0.5744, -0.9273, -0.4159, -0.4318,  0.1391,  0.8793, -1.8171,
          1.7217,  0.3396, -0.6089, -1.0367,  1.0209,  0.3137, -0.3976, -0.7220,
         -0.2993,  0.9837,  0.4121,  0.3082]])
tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, 

出力には，aとxの２つのサンプルがあります．

aが今回計算したサンプルで，xについては，引数として与えたサンプルがそのまま入っています．

次に，尤度計算をします．

尤度計算では， 全ての変数のデータを与える必要があります．

aについては，上記でサンプルした値を入れて計算しましょう．

In [49]:
_sample2 = samples['a']
log_like = p2.log_prob()
print(log_like.eval({'x': _sample1, 'a': _sample2}))
print_latex(log_like)

tensor([-15.8274], grad_fn=<SumBackward2>)


<IPython.core.display.Math object>

なお，これはもっと簡単に書くことができます．

上記のサンプリングで全ての変数とその値がdict形式で出力されたので，それをそのままlog_likelihoodの引数とすればいいのです．

In [50]:
print(log_like.eval(samples))

tensor([-15.8274], grad_fn=<SumBackward2>)


このように記述できる利点は， **変数の数が増えても同じ書き方で尤度計算を実行できる**ことです．

サンプリング->尤度計算という処理は，深層生成モデルでは数多く登場します． このように記述できることで，どのような確率分布の形であっても容易に尤度を計算できるようになるのです．

## 2.3 確率分布の積の設定

最初に書いたとおり，Pixyzの特徴の1つは，確率分布の積を簡単に記述できることです．

ここで，これまで設定した確率分布を再度確認しましょう．

In [53]:
print(p1) 
print_latex(p1)

Distribution:
  p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

In [54]:
print(p2)
print_latex(p2)

Distribution:
  p(a|x)
Network architecture:
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

数式的には，これらを掛け合わせることで同時分布を定義できます． 

$p(x,a) = p(a|x)p(x)$

では，Pixyzではこれをどのように記述するのでしょうか．

実は，それぞれの確率分布のインスタンスを**文字通り掛け合わせるだけ**でいいのです．

In [55]:
p3 = p1 * p2

p3ではどのような確率分布が定義されているか確認しましょう．

In [56]:
print(p3)
print_latex(p3)

Distribution:
  p(a,x) = p(a|x)p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

確かに，同時分布が定義されていることがわかります．

このインスタンスp3からも，これまでと同様にサンプリングや尤度計算ができます．

In [59]:
samples = p3.sample()
log_like = p3.log_prob().eval(samples)
print(log_like)
print_latex(p3.log_prob())

tensor([-82.8117], grad_fn=<AddBackward0>)


<IPython.core.display.Math object>

In [ ]:
## どういうこと？？

この例をみてわかるように，サンプリングや尤度計算において，もはやp3を構成する各分布の形を気にする必要はありません．

このような記述方法は，Pythonにおける確率モデリングライブラリでは**Pixyzが初めて採用しました**．

これは，確率分布が増えても同じです． 例えば，次の確率分布を定義しましょう．

In [60]:
class P4(Normal):
    def __init__(self):
        super(P4, self).__init__(cond_var=["a", "x"], var=["y"])

        self.fc1 = nn.Linear(x_dim, 10)
        self.fc2 = nn.Linear(a_dim, 10)
        self.fc21 = nn.Linear(10+10, 20)
        self.fc22 = nn.Linear(10+10, 20)

    def forward(self, a, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(a))
        h12 = torch.cat([h1, h2], 1)
        return {"loc":self.fc21(h12), "scale":F.softplus(self.fc22(h12))}
    
p4 = P4()

print(p4)
print_latex(p4)

Distribution:
  p(y|a,x)
Network architecture:
  P4(
    name=p, distribution_name=Normal,
    var=['y'], cond_var=['a', 'x'], input_var=['a', 'x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc2): Linear(in_features=20, out_features=10, bias=True)
    (fc21): Linear(in_features=20, out_features=20, bias=True)
    (fc22): Linear(in_features=20, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

p4は，aとxで条件付けられた確率分布です．他に$p(a|x)$，$p(x)$をつかって，同時分布は以下のように書けます．

$p(y,a,x)=p(y|a,x)p(a|x)p(x)$

同時分布が3つの確率分布の積になっていますが，これも上記と同様に書けます．

In [62]:
p5 = p4 * p2 * p1
print(p5)
print_latex(p5)

Distribution:
  p(y,a,x) = p(y|a,x)p(a|x)p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )
  P4(
    name=p, distribution_name=Normal,
    var=['y'], cond_var=['a', 'x'], input_var=['a', 'x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc2): Linear(in_features=20, out_features=10, bias=True)
    (fc21): Linear(in_features=20, out_features=20, bias=True)
    (fc22): Linear(in_features=20, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

distributionをみると，p5がどのような分布から構成されているかを表示します． サンプリングや尤度計算も全く同じようにできます．

In [65]:
samples = p5.sample()
log_like = p5.log_prob().eval(samples)
print(log_like)
print_latex(p5.log_prob())

tensor([-109.5795], grad_fn=<AddBackward0>)


<IPython.core.display.Math object>

このように，分布や深層ニューラルネットワークの形を気にせずに，同じ記述方法でサンプリングや尤度計算ができます．
